<a href="https://colab.research.google.com/github/Loyloyy/Hotel_Cancellation_Prediction/blob/master/Hotel_Analysis(Tensorflow_Raw)_%3CWork_in_progress%3E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import io
import tensorflow as tf
%tensorflow_version 2.x
import functools
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, auc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import files
uploaded = files.upload()

Saving hotel_bookings.csv to hotel_bookings.csv


In [0]:
hotel_data = pd.read_csv(io.BytesIO(uploaded['hotel_bookings.csv']))

In [0]:
df_test = pd.read_csv(io.BytesIO(uploaded['hotel_bookings.csv']))

float_cols = [c for c in df_test if df_test[c].dtype == "float32"]
float64_cols = {c: np.float64 for c in float_cols}

hotel_data = pd.read_csv(io.BytesIO(uploaded['hotel_bookings.csv']), engine='c', dtype=float64_cols)

In [0]:
final_data = hotel_data.copy()

In [0]:
le = preprocessing.LabelEncoder()

# Dropping the 'company' column
final_data = final_data.drop(['company'], axis = 1)

# Remove the null values in the country columns

indices = final_data.loc[pd.isna(final_data['country']), :].index  
final_data = final_data.drop(final_data.index[indices])  

final_data['country'] = le.fit_transform(final_data['country']) 

final_data = pd.get_dummies(data = final_data, columns = ['hotel','meal', 'market_segment'
                                                            , 'distribution_channel', 'reserved_room_type', 'deposit_type'
                                                            , 'assigned_room_type', 'customer_type', 'reservation_status'])

final_data = final_data.drop(columns = [ 'reservation_status_Canceled', 'reservation_status_Check-Out'
                                          , 'reservation_status_No-Show', 'arrival_date_day_of_month'
                                          , 'stays_in_weekend_nights', 'children', 'arrival_date_week_number'
                                          , 'arrival_date_week_number', 'arrival_date_month', 'agent'
                                          , 'reservation_status_date'], axis = 1)

label = final_data['is_canceled']


final_data = final_data.drop(['is_canceled'], axis =1)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(final_data, label, test_size = 0.20, random_state = 42)

## Setting up the model parameters

In [0]:
num_classes = 2 # total number of outputs (in this case, cancelled or not called, hence 2)
num_features = 63 # data features (you can check for the input size by using the shape function, in this case, 63)

# Training parameters.
learning_rate = 0.00001
training_steps = 63000
batch_size = 63
display_step = 315

# Network parameters.
n_hidden_1 = 250 # 1st layer number of neurons.
n_hidden_2 = 250 # 2nd layer number of neurons.
n_hidden_3 = 250 # 3rd layer number of neurons.
n_hidden_4 = 250 # 4th layer number of neurons.

In [0]:
# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [0]:
# Store layers weight & bias

# A random value generator to initialize weights.
random_normal = tf.initializers.RandomNormal()

weights = {
    'h1': tf.Variable(random_normal([num_features, n_hidden_1])),
    'h2': tf.Variable(random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(random_normal([n_hidden_2, n_hidden_3])),
    'h4': tf.Variable(random_normal([n_hidden_3, n_hidden_4])),
    'out': tf.Variable(random_normal([n_hidden_4, num_classes])),
}
biases = {
    'b1': tf.Variable(random_normal([n_hidden_1])),
    'b2': tf.Variable(random_normal([n_hidden_2])),
    'b3': tf.Variable(random_normal([n_hidden_3])),
    'b4': tf.Variable(random_normal([n_hidden_4])),
    'out': tf.Variable(random_normal([num_classes])),
}

In [0]:
# Create model.
def neural_net(x):
    
    x = tf.cast(x, dtype=tf.float32)

    # Hidden fully connected layer with 250 neurons.
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Apply sigmoid to layer_1 output for non-linearity.
    layer_1 = tf.nn.relu(layer_1)
    
    # Hidden fully connected layer with 250 neurons.
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Apply sigmoid to layer_2 output for non-linearity.
    layer_2 = tf.nn.relu(layer_2)

    # Hidden fully connected layer with 250 neurons.
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    # Apply sigmoid to layer_2 output for non-linearity.
    layer_3 = tf.nn.relu(layer_3)

    # Hidden fully connected layer with 250 neurons.
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    # Apply sigmoid to layer_2 output for non-linearity.
    layer_4 = tf.nn.relu(layer_4)
    
    # Output fully connected layer with a neuron for each class.
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out']
    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(out_layer)

In [0]:
# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [0]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = neural_net(x)
        loss = cross_entropy(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = list(weights.values()) + list(biases.values())

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [31]:
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)
    
    # print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))
    
    if step % display_step == 0:
        pred = neural_net(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 315, loss: 24.849840, accuracy: 0.841270
step: 630, loss: 19.542891, accuracy: 0.841270
step: 945, loss: 20.114689, accuracy: 0.841270
step: 1260, loss: 22.740757, accuracy: 0.825397
step: 1575, loss: 19.946774, accuracy: 0.825397
step: 1890, loss: 20.673399, accuracy: 0.873016
step: 2205, loss: 12.468659, accuracy: 0.904762
step: 2520, loss: 28.575363, accuracy: 0.746032
step: 2835, loss: 18.207811, accuracy: 0.857143
step: 3150, loss: 17.961235, accuracy: 0.888889
step: 3465, loss: 27.867760, accuracy: 0.746032
step: 3780, loss: 21.116600, accuracy: 0.841270
step: 4095, loss: 28.174376, accuracy: 0.730159
step: 4410, loss: 24.211676, accuracy: 0.825397
step: 4725, loss: 20.794285, accuracy: 0.825397
step: 5040, loss: 22.719667, accuracy: 0.809524
step: 5355, loss: 16.663065, accuracy: 0.857143
step: 5670, loss: 26.199787, accuracy: 0.777778
step: 5985, loss: 23.347359, accuracy: 0.825397
step: 6300, loss: 20.400810, accuracy: 0.857143
step: 6615, loss: 18.248543, accuracy: 0.90

In [32]:
# Test model on validation set.
pred = neural_net(x_test)
print("Test Accuracy: %f" % accuracy(pred, y_test))

Test Accuracy: 0.833228


In [0]:
# Visualize predictions.
import matplotlib.pyplot as plt